## Demostração Blog DP6

Importando pacotes necessários:

In [1]:
# !pip install git+https://github.com/DP6/Marketing-Attribution-Models.git@v1.0.8

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from marketing_attribution_models import MAM

In [3]:
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pd.set_option("max_colwidth", None)

Para está demonstração, optamos por demonstrar utilizando uma base de dados bastante conhecida pelos profissionais do Marketing: Google Merchandise Store. Está base foi extraída da conta de demonstração do Google Analytics, que pode ser acessada por qualquer usuário do Google.

In [4]:
!head -n 2 /home/luanfernandes/Downloads/sessions.csv

user_pseudo_id,session_id,event_time,user_id,is_conversion,page_referrer,page_location,utm_source,utm_medium,utm_campaign,utm_term,source_medium,partition_date
1087030895.16265,1642691807-1087030895.1626450141,2022-01-20 15:16:48,null,0,https://www.buser.com.br/,https://www.buser.com.br/,(direct),(none),(none),(none),direct,2022-01-20


In [5]:
!head -n 2 /home/luanfernandes/Downloads/sessions-2.csv

user_pseudo_id,session_id,event_time,user_id,is_conversion,utm_source,utm_medium,utm_campaign,utm_term,source_medium,partition_date
1472521791.1630330703,1640456827-1472521791.1630330703,2021-12-25T18:28:18.000+0000,null,false,google,cpc,institucional,(not provided),google_ads,2021-12-25T00:00:00.000+0000


In [6]:
np.datetime64

numpy.datetime64

In [7]:
df = pd.read_csv(
    "/home/luanfernandes/Downloads/sessions-full.csv",
    dtype={
        "user_pseudo_id": str,
        "session_id": str,
#         "event_time": np.datetime64,
        "user_id": str,
        "is_conversion": bool,
        "source_medium": str,
        "partition_date": str,
    },
    parse_dates=True
).drop_duplicates(subset=['session_id'])

df = df.drop(
    columns=[
#         "page_referrer",
#         "page_location",
        "utm_source",
        "utm_medium",
        "utm_campaign",
        "utm_term",
    ]
)

In [8]:
users = [
#     "1472526294.1645135142",
    "1317354717.1641321280",
#     "679601293.1636465111"
]


df = df[df.user_pseudo_id.isin(users)].sample(frac=1.0)

# df = (
#     pd.concat(
#         [
#             df[df.user_pseudo_id.isin(users)],
#             df[~df.user_pseudo_id.isin(users)].sample(10000)
#         ]
#     )
#     .sort_values(by=['user_pseudo_id', 'event_time'])
# )

(
    df
#     .sort_values(by=['user_pseudo_id', 'event_time'])
    .head(20)
)

,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date
167226,1317354717.1641321280,1642690358-1317354717.1641321280,2022-01-20T14:42:04.000Z,NaN,False,direct,2022-01-20T00:00:00.000Z
167267,1317354717.1641321280,1647818562-1317354717.1641321280,2022-03-20T23:12:05.000Z,NaN,False,direct,2022-03-20T00:00:00.000Z
167274,1317354717.1641321280,1648315850-1317354717.1641321280,2022-03-26T17:20:14.000Z,NaN,False,direct,2022-03-26T00:00:00.000Z
167217,1317354717.1641321280,1641404009-1317354717.1641321280,2022-01-05T17:22:56.000Z,NaN,True,direct,2022-01-05T00:00:00.000Z
167261,1317354717.1641321280,1646681362-1317354717.1641321280,2022-03-07T19:18:47.000Z,NaN,False,direct,2022-03-07T00:00:00.000Z
167215,1317354717.1641321280,1641321275-1317354717.1641321280,2022-01-04T18:24:03.000Z,NaN,True,direct,2022-01-04T00:00:00.000Z
167278,1317354717.1641321280,1648642569-1317354717.1641321280,2022-03-30T12:05:34.000Z,NaN,True,direct,2022-03-30T00:00:00.000Z
167227,1317354717.1641321280,1642773235-1317354717.1641321280,2022-01-21T13:43:32.000Z,NaN,False,direct,2022-01-21T00:00:00.000Z
167241,1317354717.1641321280,1645111959-1317354717.1641321280,2022-02-17T15:22:04.000Z,NaN,False,direct,2022-02-17T00:00:00.000Z
167245,1317354717.1641321280,1645137324-1317354717.1641321280,2022-02-17T22:24:48.000Z,NaN,False,direct,2022-02-17T00:00:00.000Z


In [9]:
df[df.is_conversion]

,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date
167217,1317354717.1641321280,1641404009-1317354717.1641321280,2022-01-05T17:22:56.000Z,NaN,True,direct,2022-01-05T00:00:00.000Z
167215,1317354717.1641321280,1641321275-1317354717.1641321280,2022-01-04T18:24:03.000Z,NaN,True,direct,2022-01-04T00:00:00.000Z
167278,1317354717.1641321280,1648642569-1317354717.1641321280,2022-03-30T12:05:34.000Z,NaN,True,direct,2022-03-30T00:00:00.000Z
167269,1317354717.1641321280,1647955970-1317354717.1641321280,2022-03-22T13:22:13.000Z,NaN,True,direct,2022-03-22T00:00:00.000Z
167264,1317354717.1641321280,1647456001-1317354717.1641321280,2022-03-16T18:29:23.000Z,NaN,True,direct,2022-03-16T00:00:00.000Z
167220,1317354717.1641321280,1642001420-1317354717.1641321280,2022-01-12T15:19:46.000Z,NaN,True,direct,2022-01-12T00:00:00.000Z
167224,1317354717.1641321280,1642585296-1317354717.1641321280,2022-01-19T09:31:07.000Z,NaN,True,direct,2022-01-19T00:00:00.000Z
167232,1317354717.1641321280,1644879259-1317354717.1641321280,2022-02-14T22:43:40.000Z,NaN,True,direct,2022-02-14T00:00:00.000Z
167222,1317354717.1641321280,1642502214-1317354717.1641321280,2022-01-18T10:26:21.000Z,NaN,True,direct,2022-01-18T00:00:00.000Z
167280,1317354717.1641321280,1650453802-1317354717.1641321280,2022-04-20T11:23:27.000Z,NaN,True,direct,2022-04-20T00:00:00.000Z


In [10]:
df.shape, df.drop_duplicates().shape, df.session_id.nunique()

((66, 7), (66, 7), 66)

In [11]:
_df = df.drop_duplicates(subset=['session_id'])

_df[_df.duplicated(subset=['session_id'])]

,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date


In [12]:
(
    df[
        df.session_id.isin(df[df.duplicated(subset=['session_id'])].session_id)
    ]
)

,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date


In [13]:
# Selecionando colunas necessárias
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 167226 to 167242
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_pseudo_id  66 non-null     object
 1   session_id      66 non-null     object
 2   event_time      66 non-null     object
 3   user_id         0 non-null      object
 4   is_conversion   66 non-null     bool  
 5   source_medium   66 non-null     object
 6   partition_date  66 non-null     object
dtypes: bool(1), object(6)
memory usage: 3.7+ KB


Assim, tratamos a coluna de data para o formato correto, tratamos os valores nulos e criamos uma coluna indicando se houve ou não uma conversão naquela sessão.

Agora que tratamos nossa base, podemos criar o nosso objeto MAM, como nossa base está na granularidade de sessão por usuário e não possuímos um ID de agrupamento de jornada, é importante que passamos como True os parametros group_channels e create_journey_id_based_on_conversion.

In [14]:
DP_tribution = MAM(
    df,
    attribution_window=30,
    channels_colname="source_medium",
    group_channels=True,
    group_channels_by_id_list=["user_pseudo_id"],
    group_timestamp_colname="event_time",
    journey_with_conv_colname="is_conversion",
    create_journey_id_based_on_conversion=True,
)

DP_tribution.as_pd_dataframe()

,journey_id,journey_rnk,user_pseudo_id,channels_agg,sessions_agg,converted_agg,conversion_value,conversion_time,session_id,time_till_conv_agg
0,id:1317354717.1641321280_J:0,0,1317354717.1641321280,direct,[1641321275-1317354717.1641321280],True,1,2022-01-04 18:24:03+00:00,1641321275-1317354717.1641321280,0.0
1,id:1317354717.1641321280_J:1,1,1317354717.1641321280,direct > direct,"[1641371208-1317354717.1641321280, 1641404009-1317354717.1641321280]",True,1,2022-01-05 17:22:56+00:00,1641404009-1317354717.1641321280,9.1114 > 0.0
2,id:1317354717.1641321280_J:2,2,1317354717.1641321280,direct > direct,"[1641513040-1317354717.1641321280, 1641918053-1317354717.1641321280]",True,1,2022-01-11 16:10:19+00:00,1641918053-1317354717.1641321280,112.5036 > 0.0
3,id:1317354717.1641321280_J:3,3,1317354717.1641321280,direct,[1642001420-1317354717.1641321280],True,1,2022-01-12 15:19:46+00:00,1642001420-1317354717.1641321280,0.0
4,id:1317354717.1641321280_J:4,4,1317354717.1641321280,direct > direct,"[1642436750-1317354717.1641321280, 1642502214-1317354717.1641321280]",True,1,2022-01-18 10:26:21+00:00,1642502214-1317354717.1641321280,18.1847 > 0.0
5,id:1317354717.1641321280_J:5,5,1317354717.1641321280,direct > direct,"[1642537207-1317354717.1641321280, 1642585296-1317354717.1641321280]",True,1,2022-01-19 09:31:07+00:00,1642585296-1317354717.1641321280,13.3592 > 0.0
6,id:1317354717.1641321280_J:6,6,1317354717.1641321280,direct,[1642674607-1317354717.1641321280],True,1,2022-01-20 10:19:32+00:00,1642674607-1317354717.1641321280,0.0
7,id:1317354717.1641321280_J:7,7,1317354717.1641321280,direct > direct > direct > direct > direct > direct > direct,"[1642690358-1317354717.1641321280, 1642773235-1317354717.1641321280, 1642776734-1317354717.1641321280, 1642850087-1317354717.1641321280, 1643197940-1317354717.1641321280, 1644872883-1317354717.1641321280, 1644879259-1317354717.1641321280]",True,1,2022-02-14 22:43:40+00:00,1644879259-1317354717.1641321280,608.0267 > 585.0022 > 584.0333 > 563.6575 > 467.0311 > 1.7697 > 0.0
8,id:1317354717.1641321280_J:8,8,1317354717.1641321280,direct > direct > direct > direct > direct,"[1644926125-1317354717.1641321280, 1644929331-1317354717.1641321280, 1644966902-1317354717.1641321280, 1645004932-1317354717.1641321280, 1645047212-1317354717.1641321280]",True,1,2022-02-16 21:22:56+00:00,1645047212-1317354717.1641321280,33.635 > 32.7447 > 22.3083 > 11.7444 > 0.0
9,id:1317354717.1641321280_J:9,9,1317354717.1641321280,direct > direct > direct > direct > direct > direct > direct > direct > direct,"[1645094629-1317354717.1641321280, 1645104643-1317354717.1641321280, 1645107781-1317354717.1641321280, 1645111959-1317354717.1641321280, 1645116384-1317354717.1641321280, 1645120769-1317354717.1641321280, 1645120770-1317354717.1641321280, 1645137324-1317354717.1641321280, 1645203625-1317354717.1641321280]",True,1,2022-02-18 16:49:50+00:00,1645203625-1317354717.1641321280,30.2764 > 27.4947 > 26.6236 > 25.4628 > 24.2347 > 23.0167 > 23.0158 > 18.4172 > 0.0


In [15]:
DP_tribution.DataFrame

,journey_id,journey_rnk,user_pseudo_id,channels_agg,sessions_agg,converted_agg,conversion_value,conversion_time,session_id,time_till_conv_agg
0,id:1317354717.1641321280_J:0,0,1317354717.1641321280,direct,[1641321275-1317354717.1641321280],True,1,2022-01-04 18:24:03+00:00,1641321275-1317354717.1641321280,0.0
1,id:1317354717.1641321280_J:1,1,1317354717.1641321280,direct > direct,"[1641371208-1317354717.1641321280, 1641404009-1317354717.1641321280]",True,1,2022-01-05 17:22:56+00:00,1641404009-1317354717.1641321280,9.1114 > 0.0
2,id:1317354717.1641321280_J:2,2,1317354717.1641321280,direct > direct,"[1641513040-1317354717.1641321280, 1641918053-1317354717.1641321280]",True,1,2022-01-11 16:10:19+00:00,1641918053-1317354717.1641321280,112.5036 > 0.0
3,id:1317354717.1641321280_J:3,3,1317354717.1641321280,direct,[1642001420-1317354717.1641321280],True,1,2022-01-12 15:19:46+00:00,1642001420-1317354717.1641321280,0.0
4,id:1317354717.1641321280_J:4,4,1317354717.1641321280,direct > direct,"[1642436750-1317354717.1641321280, 1642502214-1317354717.1641321280]",True,1,2022-01-18 10:26:21+00:00,1642502214-1317354717.1641321280,18.1847 > 0.0
5,id:1317354717.1641321280_J:5,5,1317354717.1641321280,direct > direct,"[1642537207-1317354717.1641321280, 1642585296-1317354717.1641321280]",True,1,2022-01-19 09:31:07+00:00,1642585296-1317354717.1641321280,13.3592 > 0.0
6,id:1317354717.1641321280_J:6,6,1317354717.1641321280,direct,[1642674607-1317354717.1641321280],True,1,2022-01-20 10:19:32+00:00,1642674607-1317354717.1641321280,0.0
7,id:1317354717.1641321280_J:7,7,1317354717.1641321280,direct > direct > direct > direct > direct > direct > direct,"[1642690358-1317354717.1641321280, 1642773235-1317354717.1641321280, 1642776734-1317354717.1641321280, 1642850087-1317354717.1641321280, 1643197940-1317354717.1641321280, 1644872883-1317354717.1641321280, 1644879259-1317354717.1641321280]",True,1,2022-02-14 22:43:40+00:00,1644879259-1317354717.1641321280,608.0267 > 585.0022 > 584.0333 > 563.6575 > 467.0311 > 1.7697 > 0.0
8,id:1317354717.1641321280_J:8,8,1317354717.1641321280,direct > direct > direct > direct > direct,"[1644926125-1317354717.1641321280, 1644929331-1317354717.1641321280, 1644966902-1317354717.1641321280, 1645004932-1317354717.1641321280, 1645047212-1317354717.1641321280]",True,1,2022-02-16 21:22:56+00:00,1645047212-1317354717.1641321280,33.635 > 32.7447 > 22.3083 > 11.7444 > 0.0
9,id:1317354717.1641321280_J:9,9,1317354717.1641321280,direct > direct > direct > direct > direct > direct > direct > direct > direct,"[1645094629-1317354717.1641321280, 1645104643-1317354717.1641321280, 1645107781-1317354717.1641321280, 1645111959-1317354717.1641321280, 1645116384-1317354717.1641321280, 1645120769-1317354717.1641321280, 1645120770-1317354717.1641321280, 1645137324-1317354717.1641321280, 1645203625-1317354717.1641321280]",True,1,2022-02-18 16:49:50+00:00,1645203625-1317354717.1641321280,30.2764 > 27.4947 > 26.6236 > 25.4628 > 24.2347 > 23.0167 > 23.0158 > 18.4172 > 0.0


Acessando o atributo .DataFrame, conseguimos observar que houve uma mudança significativa em nossa base de dados. Observamos que a granularidade da tabela foi alterada de sessão para jornada e que há uma coluna nova de id da jornada.

In [16]:
df.shape

(66, 7)

In [17]:
DP_tribution.attribution_first_click()
DP_tribution.attribution_last_click()
DP_tribution.attribution_linear()
DP_tribution.attribution_time_decay()

markov_results = DP_tribution.attribution_markov(transition_to_same_state=False)

In [18]:
DP_tribution.group_by_channels_models

,channels,attribution_first_click_heuristic,attribution_last_click_heuristic,attribution_linear_heuristic,attribution_time_decay0.5_freq24_heuristic,attribution_markov_algorithmic
0,direct,16,16,16.000,16.000,16.000


In [19]:
_df = None
def get_intermediate(df):
    global _df
    _df = df
    return df

In [20]:
user = users[0]
print("user: ", user)

(
    df
    .query("user_pseudo_id == @user")
    .sort_values(by=['user_pseudo_id', 'event_time'])
    .head(20)
)

user:  1317354717.1641321280


,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date
167215,1317354717.1641321280,1641321275-1317354717.1641321280,2022-01-04T18:24:03.000Z,NaN,True,direct,2022-01-04T00:00:00.000Z
167216,1317354717.1641321280,1641371208-1317354717.1641321280,2022-01-05T08:16:15.000Z,NaN,False,direct,2022-01-05T00:00:00.000Z
167217,1317354717.1641321280,1641404009-1317354717.1641321280,2022-01-05T17:22:56.000Z,NaN,True,direct,2022-01-05T00:00:00.000Z
167218,1317354717.1641321280,1641513040-1317354717.1641321280,2022-01-06T23:40:06.000Z,NaN,False,direct,2022-01-06T00:00:00.000Z
167219,1317354717.1641321280,1641918053-1317354717.1641321280,2022-01-11T16:10:19.000Z,NaN,True,direct,2022-01-11T00:00:00.000Z
167220,1317354717.1641321280,1642001420-1317354717.1641321280,2022-01-12T15:19:46.000Z,NaN,True,direct,2022-01-12T00:00:00.000Z
167221,1317354717.1641321280,1642436750-1317354717.1641321280,2022-01-17T16:15:16.000Z,NaN,False,direct,2022-01-17T00:00:00.000Z
167222,1317354717.1641321280,1642502214-1317354717.1641321280,2022-01-18T10:26:21.000Z,NaN,True,direct,2022-01-18T00:00:00.000Z
167223,1317354717.1641321280,1642537207-1317354717.1641321280,2022-01-18T20:09:34.000Z,NaN,False,direct,2022-01-18T00:00:00.000Z
167224,1317354717.1641321280,1642585296-1317354717.1641321280,2022-01-19T09:31:07.000Z,NaN,True,direct,2022-01-19T00:00:00.000Z


In [21]:
# journeys
(
    DP_tribution.DataFrame
    .query("user_pseudo_id == @user")
    .sort_values(by=['user_pseudo_id', 'conversion_time'])
#     [['time_till_conv_agg', 'channels_agg'] + [c for c in DP_tribution.DataFrame.columns if 'attribution' in c]]
    .head(5)
)

,journey_id,journey_rnk,user_pseudo_id,channels_agg,sessions_agg,converted_agg,conversion_value,conversion_time,session_id,time_till_conv_agg,attribution_first_click_heuristic,attribution_last_click_heuristic,attribution_linear_heuristic,attribution_time_decay0.5_freq24_heuristic,attribution_markov_algorithmic
0,id:1317354717.1641321280_J:0,0,1317354717.1641321280,direct,[1641321275-1317354717.1641321280],True,1,2022-01-04 18:24:03+00:00,1641321275-1317354717.1641321280,0.0,1,1,1.0,1.0,1.0
1,id:1317354717.1641321280_J:1,1,1317354717.1641321280,direct > direct,"[1641371208-1317354717.1641321280, 1641404009-1317354717.1641321280]",True,1,2022-01-05 17:22:56+00:00,1641404009-1317354717.1641321280,9.1114 > 0.0,1 > 0,0 > 1,0.5 > 0.5,0.5 > 0.5,0.5 > 0.5
2,id:1317354717.1641321280_J:2,2,1317354717.1641321280,direct > direct,"[1641513040-1317354717.1641321280, 1641918053-1317354717.1641321280]",True,1,2022-01-11 16:10:19+00:00,1641918053-1317354717.1641321280,112.5036 > 0.0,1 > 0,0 > 1,0.5 > 0.5,0.058823529411764705 > 0.9411764705882353,0.5 > 0.5
3,id:1317354717.1641321280_J:3,3,1317354717.1641321280,direct,[1642001420-1317354717.1641321280],True,1,2022-01-12 15:19:46+00:00,1642001420-1317354717.1641321280,0.0,1,1,1.0,1.0,1.0
4,id:1317354717.1641321280_J:4,4,1317354717.1641321280,direct > direct,"[1642436750-1317354717.1641321280, 1642502214-1317354717.1641321280]",True,1,2022-01-18 10:26:21+00:00,1642502214-1317354717.1641321280,18.1847 > 0.0,1 > 0,0 > 1,0.5 > 0.5,0.5 > 0.5,0.5 > 0.5


In [22]:
DP_tribution.original_df.shape

(66, 14)

In [23]:
(
    DP_tribution.original_df
    .query("user_pseudo_id == @user")
    .sort_values(by=['user_pseudo_id', 'event_time'])
    .head(50)
)

,index,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date,time_till_conv,attribution_first_click_heuristic,attribution_last_click_heuristic,attribution_linear_heuristic,attribution_time_decay0.5_freq24_heuristic,attribution_markov_algorithmic
0,167215,1317354717.1641321280,1641321275-1317354717.1641321280,2022-01-04 18:24:03+00:00,NaN,True,direct,2022-01-04T00:00:00.000Z,0.000,1,1,1.000,1.000,1.000
1,167216,1317354717.1641321280,1641371208-1317354717.1641321280,2022-01-05 08:16:15+00:00,NaN,False,direct,2022-01-05T00:00:00.000Z,9.111,1,0,0.500,0.500,0.500
2,167217,1317354717.1641321280,1641404009-1317354717.1641321280,2022-01-05 17:22:56+00:00,NaN,True,direct,2022-01-05T00:00:00.000Z,0.000,0,1,0.500,0.500,0.500
3,167218,1317354717.1641321280,1641513040-1317354717.1641321280,2022-01-06 23:40:06+00:00,NaN,False,direct,2022-01-06T00:00:00.000Z,112.504,1,0,0.500,0.059,0.500
4,167219,1317354717.1641321280,1641918053-1317354717.1641321280,2022-01-11 16:10:19+00:00,NaN,True,direct,2022-01-11T00:00:00.000Z,0.000,0,1,0.500,0.941,0.500
5,167220,1317354717.1641321280,1642001420-1317354717.1641321280,2022-01-12 15:19:46+00:00,NaN,True,direct,2022-01-12T00:00:00.000Z,0.000,1,1,1.000,1.000,1.000
6,167221,1317354717.1641321280,1642436750-1317354717.1641321280,2022-01-17 16:15:16+00:00,NaN,False,direct,2022-01-17T00:00:00.000Z,18.185,1,0,0.500,0.500,0.500
7,167222,1317354717.1641321280,1642502214-1317354717.1641321280,2022-01-18 10:26:21+00:00,NaN,True,direct,2022-01-18T00:00:00.000Z,0.000,0,1,0.500,0.500,0.500
8,167223,1317354717.1641321280,1642537207-1317354717.1641321280,2022-01-18 20:09:34+00:00,NaN,False,direct,2022-01-18T00:00:00.000Z,13.359,1,0,0.500,0.500,0.500
9,167224,1317354717.1641321280,1642585296-1317354717.1641321280,2022-01-19 09:31:07+00:00,NaN,True,direct,2022-01-19T00:00:00.000Z,0.000,0,1,0.500,0.500,0.500


In [24]:
# by sessions
n = 1 # number of conversion
(
    DP_tribution.original_df
    .query("user_pseudo_id == @user")
    .pipe(get_intermediate)
    [
#         (_df.event_time > _df[_df.is_conversion].event_time.iloc[n])
#         &
        (_df.event_time <= _df[_df.is_conversion].event_time.iloc[n])
    ]
)

,index,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date,time_till_conv,attribution_first_click_heuristic,attribution_last_click_heuristic,attribution_linear_heuristic,attribution_time_decay0.5_freq24_heuristic,attribution_markov_algorithmic
0,167215,1317354717.1641321280,1641321275-1317354717.1641321280,2022-01-04 18:24:03+00:00,NaN,True,direct,2022-01-04T00:00:00.000Z,0.000,1,1,1.000,1.000,1.000
1,167216,1317354717.1641321280,1641371208-1317354717.1641321280,2022-01-05 08:16:15+00:00,NaN,False,direct,2022-01-05T00:00:00.000Z,9.111,1,0,0.500,0.500,0.500
2,167217,1317354717.1641321280,1641404009-1317354717.1641321280,2022-01-05 17:22:56+00:00,NaN,True,direct,2022-01-05T00:00:00.000Z,0.000,0,1,0.500,0.500,0.500


In [25]:
df.iloc[251209-10:251209+20, :]

,user_pseudo_id,session_id,event_time,user_id,is_conversion,source_medium,partition_date


In [26]:
l1, l2 = df.query("user_pseudo_id == @user").event_time.min(), df.query("user_pseudo_id == @user").event_time.max()
(
    df
    [
        df.user_pseudo_id.isin(
            df[(df.event_time >= l1) & (df.event_time <= l2)].user_pseudo_id
        )
    ]
).user_pseudo_id.nunique()

1

In [27]:
idx = DP_tribution.original_df.query("user_pseudo_id == @user").index

In [28]:
len(DP_tribution.channels)

16

In [29]:
channels_value = DP_tribution.channels.apply(
            lambda channels: np.asarray(([0] * (len(channels) - 1)) + [1])
        )

print(len(DP_tribution.original_df))
len(channels_value.explode().reset_index(drop=True))

66


66